## Initialize

In [106]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import requests



from PyPDF2 import PdfFileReader

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()



## Choose file folder

In [2]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
options.remove('notebooks')
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Select:', options=('externDocuments', 'internDocuments'), value=None)

## Take all files from folder

In [107]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            filename = filename.replace('.pdf', '')

            # encode to base64
            encoded_string = base64.b64encode(f.read())
            encoded_doc = encoded_string.decode('ascii')

            # Get file info
            pdf = PdfFileReader(f)
            info = pdf.getDocumentInfo()
            created_at = str(info.creation_date)
            created_at_string = created_at[0:10]
            changed_at = str(info.modification_date)
            changed_at_string = changed_at[0:10]
            print(filename + ' Created at: ' + created_at_string +
                  ' Changed at: ' + changed_at_string)
            if info.author == None:
                author = 'Not specified'
            else:
                author = info.author

            #get image
            # pdf_file = fitz.open(os.path.join(current_path, filename) + '.pdf')
            # page = pdf_file[0]
            # image_list = page.get_images()
            
            # for image_index, img in enumerate(page.get_images(), start=0):
            #     xref = img[0]
                
            #     base_image = pdf_file.extract_image(xref)
            #     image_bytes = base_image['image']
            #     stream = io.BytesIO(image_bytes)
            #     img2 = Image.open(stream)
            #     # img2.size(50,50)
            #     print(img2.height, img2.width)
            #     img2.show()
            #     image_sequence = img2.getdata()
            #     # image_array = np(image_sequence)
            #     image_array = np.asarray(image_sequence)
            #     image_Ext = base_image['ext']
            #     break

            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {   
                        'title': filename,
                        'author': author,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)

else:
    print('No folder selected for PDF files')


Xref table not zero-indexed. ID numbers for objects will be corrected.


handbook on implementation of the services directive-gp_eudor_WEB_KM7807096ENC_002 Created at: 2007-12-17 Changed at: 2008-01-21
Implementation Report 2020 Created at: 2020-05-11 Changed at: 2020-05-11
SWD 2012-148 implementation SD Created at: 2012-06-08 Changed at: 2012-06-08
user guide PQD EN Created at: 2020-07-16 Changed at: 2022-09-14


## Post data to server

In [108]:
for element in payload_list:
    response = requests.request("POST", f'{base_url}/v1/data/DocResearchDemoExample', headers=headers, data=element)

    print(response.text)

{"Document":[{"id":"e884f93c-60e2-11ed-9f8b-88d82e07e733"}]}
{"Document":[{"id":"e9395351-60e2-11ed-85fa-88d82e07e733"}]}
{"Document":[{"id":"e971377d-60e2-11ed-b06d-88d82e07e733"}]}
{"Document":[{"id":"e9bfa3ea-60e2-11ed-9523-88d82e07e733"}]}
